In [33]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from math import pi,cos,sin
import os 
newpath = 'images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

# Creating the grid and the corresponding constants


In this section of the code the grid is constructed with some of the essential constants for the simulation.

In [34]:
nx = 150                           # Amount of grid points x direction.
ny = 60                            # "                   " y direction.
a  = 0.0004                        # Acceleration at each timestep. Should be kept low. 
time = 200                         # Amount of timesteps.
tau  = 2

# Construction of 9 direction for the use of d2q9 model and assignment of the appropriate weights.
q  = 9
c  = np.zeros((2,q), dtype='int')
c[0, 1:q] = np.round(np.cos(np.linspace(0, (2*pi - 2*pi/q), q-1)))
c[1, 1:q] = np.round(np.sin(np.linspace(0, (2*pi - 2*pi/q), q-1)))

weights = 1/9*np.ones(q)
weights[0] = 4/9
weights[2::2] = 1/36               # Sets all even weights other than 0.

# Objects 

In this section the objects that can be placed in the flow are created. You can choose to place a line or a block. The dimensions of the object can be changed and the location of the object.   

In [35]:
# Object in flow: choose 'block' for a block and 'line' for a line 
obj = 'block'
# Diminsions and location of object 
center = [nx/2, ny/2] # Center coordinates of the object 
width  = 10
length = 10
# Locations of the block/line: rectangle ABCD/line AD
locA   = np.int_([center[0] - width/2, center[1] - length/2]) # Left under coordinate
locB   = np.int_([center[0] + width/2, center[1] - length/2]) # Right under coordinate
locC   = np.int_([center[0] + width/2, center[1] + length/2]) # Right top coordinate
locD   = np.int_([center[0] - width/2, center[1] + length/2]) # Left top coordinate

# Simulation 

In this section the simulation starts. It will create every 10 time steps an image of the velocity profile in the cylinder.

In [36]:
# Initialisation of the densities ni at all points and for all directions i.
n   = np.zeros((nx,ny,q), dtype='float')                
neq = np.zeros((nx,ny,q), dtype='float')
n[:, :, 0] = 1                     # Only the 0 direction (at rest) is set to nonzero.

# Start of the time evolution algorithm.
for t in range(time):

    # All densities are moved to the appropriate neighbour by rolling the density matrix along the x and y axis when
    # appropriate. The directions stored in c indicate when this is appropriate. This also applies a periodic boundary
    # condition at the inlet and outlet.
    for i in range(1, q):
        n[:, :, i] = np.roll(n[:, :, i], c[0, i], axis = 0)
        n[:, :, i] = np.roll(n[:, :, i], c[1, i], axis = 1)

    # At the boundaries the direction are inversed by rolling the matrix (except the 0 direction) in the q direction
    # for 4 places. This implements a no slip condition by forcing the average velocity to be 0.
    n[:, 0,    1:q] = np.roll(n[:, 0,    1:q], round((q-1) / 2), axis = 1)
    n[:, ny-1, 1:q] = np.roll(n[:, ny-1, 1:q], round((q-1) / 2), axis = 1)
    # Boundaries at the object, in this case a block
    if obj == 'block':
        n[locA[0], locA[1]:locD[1], 1:q] = np.roll(n[locA[0], locA[1]:locD[1], 1:q], round((q-1)/2), axis = 1)
        n[locB[0], locB[1]:locC[1], 1:q] = np.roll(n[locB[0], locB[1]:locC[1], 1:q], round((q-1)/2), axis = 1)
        n[locA[0]:locB[0], locA[1], 1:q] = np.roll(n[locA[0]:locB[0], locA[1], 1:q], round((q-1)/2), axis = 1)
        n[locD[0]:locC[0], locD[1], 1:q] = np.roll(n[locD[0]:locC[0], locD[1], 1:q], round((q-1)/2), axis = 1)
    if obj == 'line':
        n[locA[0],locA[1]:locD[1],1:q] = np.roll(n[locA[0],locA[1]:locD[1],1:q], round((q-1)/2), axis = 1)

    # Calculation of local 'total' density and local average velocity.
    rho = np.sum(n, axis = 2)
    u = [np.sum(n*c[0, :], axis = 2), np.sum(n*c[1,:], axis = 2)] / rho

    # The average velocity is increased along the x-direction to implement the pressure driving the flow.
    u[0] += a 

    # Calculation of the equilibrium densities for each direction at each location using the formula for d2q9.
    cdotu = np.inner(c.transpose(), u.transpose()).transpose()
    udotu = u[1]**2 + u[0]**2
    for i in range(q):
        neq[:, :, i] = weights[i] * rho * (1 + 3*cdotu[:,:,i] + 9/2*cdotu[:,:,i]**2 - 3/2*udotu)

    # Relaxation of the densities towards the equilibrium distribution. This models the collision interaction in the fluid,
    # the viscous effect.
    n = n - (1/tau)*(n-neq)
    
    if (t%10 == 0): # Visualization: The image is saved every 10 time steps 
        plt.imshow(np.sqrt(u[0]**2 + u[1]**2).transpose(), cmap=cm.RdBu)
        plt.title('Lattice Boltzmann simulated flow ')
        plt.colorbar()
        plt.savefig("images/vel_line."+str(t/10).zfill(4)+".png")
        plt.clf()